In [1]:
import json

only_gt_data_path = "/data/aofei/hallucination/Slake/data/training_gt_bboxes.json"
with open(only_gt_data_path, 'r') as file:
    gt_data = json.load(file)

training_path = "/data/aofei/hallucination/Slake/data/training_masks.json"
with open(training_path, 'r') as file:
    training_data = json.load(file)

In [2]:
gt_data[0], training_data[0], len(gt_data), len(training_data)

({'image': 'xmlab1/source.jpg',
  'id': 0,
  'location': 'Abdomen',
  'conversations': [{'from': 'human',
    'value': '<image>\nWhat modality is used to take this image?'},
   {'from': 'gpt', 'value': 'MRI'}],
  'bboxes': [],
  'bboxes_dict': {}},
 {'image': 'xmlab1/source.jpg',
  'id': 0,
  'conversations': [{'from': 'human',
    'value': '<image>\nWhat modality is used to take this image?'},
   {'from': 'gpt', 'value': 'MRI'}],
  'bboxes': [[1, 52, 238, 159],
   [5, 59, 217, 134],
   [115, 78, 75, 67],
   [61, 156, 26, 31]],
  'masks': []},
 4919,
 4919)

In [3]:
import os
from PIL import Image

for i in range(len(gt_data)):
    gt_i = gt_data[i]
    gt_bboxes = gt_i['bboxes']
    training_i = training_data[i]
    image_path = os.path.join("/data/aofei/hallucination/Slake/imgs", gt_i['image'])
    with Image.open(image_path) as img:
        image_width, image_height = img.size
    new_gt_bboxes = []
    if len(gt_bboxes) == 0:
        training_data[i]['gt_bboxes'] = []
        continue
    for gt_bbox in gt_bboxes:
        bbox = gt_bbox
        # Original coordinates
        top_left_x = bbox[0]
        top_left_y = bbox[1]
        width = bbox[2]
        height = bbox[3]

        # Calculate bottom-right coordinates
        bottom_right_x = top_left_x + width
        bottom_right_y = top_left_y + height

        # Normalize coordinates
        top_left_x = int(top_left_x / image_width * 256)
        top_left_y = int(top_left_y / image_height * 256)
        bottom_right_x = int(bottom_right_x / image_width * 256)
        bottom_right_y = int(bottom_right_y / image_height * 256)
        width = bottom_right_x - top_left_x
        height = bottom_right_y - top_left_y

        # Update the coordinates
        # new_gt_bboxes.append([top_left_x, top_left_y, bottom_right_x, bottom_right_y])
        new_gt_bboxes.append([top_left_x, top_left_y, width, height])
    gt_i['bboxes'] = new_gt_bboxes
    training_data[i]['gt_bboxes'] = new_gt_bboxes

In [8]:
s = 0
import copy
only_gt = []
weak_label_data = []
for i in training_data:
    if 'gt_bboxes' in i:
        if len(i['gt_bboxes']) > 0:
            s += 1
            k, p = copy.deepcopy(i), copy.deepcopy(i)
            k['bboxes'] = i['gt_bboxes']
            del k['gt_bboxes']
            del k['masks']
            only_gt.append(k)
            del p['gt_bboxes']
            del p['masks']
            weak_label_data.append(p)

len(training_data), s

(4919, 611)

In [10]:
save_path = "/data/aofei/hallucination/Slake/data/with_gt/training_only_wgt.json"
with open(save_path, 'w') as file:
    json.dump(only_gt, file, indent=2)

save_path = "/data/aofei/hallucination/Slake/data/with_gt/training_only_weak.json"
with open(save_path, 'w') as file:
    json.dump(weak_label_data, file, indent=2)

In [3]:
test_path = "/data/aofei/hallucination/Slake/data/test.json"
with open(test_path, 'r') as file:
    test_data = json.load(file)
test_data[0], len(test_data)

({'image': 'xmlab102/source.jpg',
  'id': 11934,
  'organ': 'Lung',
  'answer_type': 'OPEN',
  'conversations': [{'from': 'human',
    'value': '<image>\nWhat modality is used to take this image?'},
   {'from': 'gpt', 'value': 'CT'}]},
 1061)

In [5]:
import os
bbox_path = os.path.join("/data/aofei/hallucination/Slake/imgs", test_data[0]['image'].split("/")[0], "detection.json")
with open(bbox_path, 'r') as file:
    bboxes = json.load(file)
bboxes

[{'Lung Cancer': [289.0, 301.0, 17.0, 22.0]}]

In [7]:
## for test data
# only_gt_data_path = "/data/aofei/hallucination/Slake/data/training_gt_bboxes.json"
# with open(only_gt_data_path, 'r') as file:
#     gt_data = json.load(file)
import os
from PIL import Image

test_path = "/data/aofei/hallucination/Slake/data/test.json"
with open(test_path, 'r') as file:
    test_data = json.load(file)


for i in range(len(test_data)):
    test_i = test_data[i]
    # test_bboxes = test_i['bboxes']
    # training_i = training_data[i]
    bbox_path = os.path.join("/data/aofei/hallucination/Slake/imgs", test_data[i]['image'].split("/")[0], "detection.json")
    with open(bbox_path, 'r') as file:
        gt_bboxes_ori = json.load(file)
    # gt_bboxes = {}
    # for gt_bbox in gt_bboxes_ori:
        
    #     gt_bboxes[gt_bbox['label']] = gt_bbox['bbox']
    # gt_bbox_keys = list(gt_bboxes.keys())
    test_question = test_i['conversations'][0]['value']
    test_answer = test_i['conversations'][1]['value']
    test_gt_bboxes = []
    for bbox in gt_bboxes_ori:
        k, v = list(bbox.items())[0]
        if k.lower() in test_question.lower() or k.lower() in test_answer.lower():
            test_gt_bboxes.append(v)

    image_path = os.path.join("/data/aofei/hallucination/Slake/imgs", test_i['image'])
    with Image.open(image_path) as img:
        image_width, image_height = img.size
    new_gt_bboxes = []
    for gt_bbox in test_gt_bboxes:
        bbox = gt_bbox
        # Original coordinates
        top_left_x = bbox[0]
        top_left_y = bbox[1]
        width = bbox[2]
        height = bbox[3]

        # Calculate bottom-right coordinates
        bottom_right_x = top_left_x + width
        bottom_right_y = top_left_y + height

        # Normalize coordinates
        top_left_x = int(top_left_x / image_width * 256)
        top_left_y = int(top_left_y / image_height * 256)
        bottom_right_x = int(bottom_right_x / image_width * 256)
        bottom_right_y = int(bottom_right_y / image_height * 256)
        width = bottom_right_x - top_left_x
        height = bottom_right_y - top_left_y

        # Update the coordinates
        # new_gt_bboxes.append([top_left_x, top_left_y, bottom_right_x, bottom_right_y])
        new_gt_bboxes.append([top_left_x, top_left_y, width, height])
    test_i['bboxes'] = new_gt_bboxes
    test_data[i]['gt_bboxes'] = new_gt_bboxes

In [8]:
test_with_gt = []
for i in test_data:
    if 'gt_bboxes' in i:
        if len(i['gt_bboxes']) > 0:
            test_with_gt.append(i)

In [9]:
test_with_gt[0], len(test_with_gt)

({'image': 'xmlab102/source.jpg',
  'id': 11941,
  'organ': 'Lung',
  'answer_type': 'OPEN',
  'conversations': [{'from': 'human',
    'value': '<image>\nWhat diseases are included in the picture?'},
   {'from': 'gpt', 'value': 'Lung Cancer'}],
  'bboxes': [[144, 150, 9, 11]],
  'gt_bboxes': [[144, 150, 9, 11]]},
 206)

In [10]:
with_gt_save_path = "/data/aofei/hallucination/Slake/data/with_gt/test_with_gt.json"
with open(with_gt_save_path, 'w') as file:
    json.dump(test_with_gt, file, indent=2)

In [12]:

#train data

train_path = "/data/aofei/hallucination/Slake/data/with_gt/training_masks_wgt.json"

with open(train_path, 'r') as file:
    train_data = json.load(file)
train_data[0], len(train_data)



({'image': 'xmlab1/source.jpg',
  'id': 0,
  'conversations': [{'from': 'human',
    'value': '<image>\nWhat modality is used to take this image?'},
   {'from': 'gpt', 'value': 'MRI'}],
  'bboxes': [[1, 52, 238, 159],
   [5, 59, 217, 134],
   [115, 78, 75, 67],
   [61, 156, 26, 31]],
  'masks': [],
  'gt_bboxes': []},
 4919)

In [13]:
## for test data
# only_gt_data_path = "/data/aofei/hallucination/Slake/data/training_gt_bboxes.json"
# with open(only_gt_data_path, 'r') as file:
#     gt_data = json.load(file)
import os
from PIL import Image

for i in range(len(train_data)):
    train_i = train_data[i]
    # test_bboxes = test_i['bboxes']
    # training_i = training_data[i]
    bbox_path = os.path.join("/data/aofei/hallucination/Slake/imgs", train_data[i]['image'].split("/")[0], "detection.json")
    with open(bbox_path, 'r') as file:
        gt_bboxes_ori = json.load(file)
    # gt_bboxes = {}
    # for gt_bbox in gt_bboxes_ori:
        
    #     gt_bboxes[gt_bbox['label']] = gt_bbox['bbox']
    # gt_bbox_keys = list(gt_bboxes.keys())
    test_question = train_i['conversations'][0]['value']
    test_answer = train_i['conversations'][1]['value']
    test_gt_bboxes = []
    for bbox in gt_bboxes_ori:
        k, v = list(bbox.items())[0]
        if k.lower() in test_question.lower() or k.lower() in test_answer.lower():
            test_gt_bboxes.append(v)

    image_path = os.path.join("/data/aofei/hallucination/Slake/imgs", test_i['image'])
    with Image.open(image_path) as img:
        image_width, image_height = img.size
    new_gt_bboxes = []
    for gt_bbox in test_gt_bboxes:
        bbox = gt_bbox
        # Original coordinates
        top_left_x = bbox[0]
        top_left_y = bbox[1]
        width = bbox[2]
        height = bbox[3]

        # Calculate bottom-right coordinates
        bottom_right_x = top_left_x + width
        bottom_right_y = top_left_y + height

        # Normalize coordinates
        top_left_x = int(top_left_x / image_width * 256)
        top_left_y = int(top_left_y / image_height * 256)
        bottom_right_x = int(bottom_right_x / image_width * 256)
        bottom_right_y = int(bottom_right_y / image_height * 256)
        width = bottom_right_x - top_left_x
        height = bottom_right_y - top_left_y

        # Update the coordinates
        # new_gt_bboxes.append([top_left_x, top_left_y, bottom_right_x, bottom_right_y])
        new_gt_bboxes.append([top_left_x, top_left_y, width, height])
    # train_i['bboxes'] = new_gt_bboxes
    train_data[i]['gt_bboxes'] = new_gt_bboxes

In [14]:
train_data[0], len(train_data)

({'image': 'xmlab1/source.jpg',
  'id': 0,
  'conversations': [{'from': 'human',
    'value': '<image>\nWhat modality is used to take this image?'},
   {'from': 'gpt', 'value': 'MRI'}],
  'bboxes': [[1, 52, 238, 159],
   [5, 59, 217, 134],
   [115, 78, 75, 67],
   [61, 156, 26, 31]],
  'masks': [],
  'gt_bboxes': []},
 4919)

In [15]:
s = 0
import copy
only_gt = []
weak_label_data = []
for i in train_data:
    if 'gt_bboxes' in i:
        if len(i['gt_bboxes']) > 0:
            s += 1
            k, p = copy.deepcopy(i), copy.deepcopy(i)
            k['bboxes'] = i['gt_bboxes']
            del k['gt_bboxes']
            del k['masks']
            only_gt.append(k)
            del p['gt_bboxes']
            del p['masks']
            weak_label_data.append(p)

len(train_data), s

(4919, 992)

In [16]:
save_path = "/data/aofei/hallucination/Slake/data/with_gt/training_only_wgt.json"
with open(save_path, 'w') as file:
    json.dump(only_gt, file, indent=2)

save_path = "/data/aofei/hallucination/Slake/data/with_gt/training_only_weak.json"
with open(save_path, 'w') as file:
    json.dump(weak_label_data, file, indent=2)

In [17]:
len(only_gt), len(weak_label_data), only_gt[0], weak_label_data[0]

(992,
 992,
 {'image': 'xmlab1/source.jpg',
  'id': 3,
  'conversations': [{'from': 'human',
    'value': '<image>\nDoes the picture contain liver?'},
   {'from': 'gpt', 'value': 'Yes'}],
  'bboxes': [[27, 53, 15, 15]]},
 {'image': 'xmlab1/source.jpg',
  'id': 3,
  'conversations': [{'from': 'human',
    'value': '<image>\nDoes the picture contain liver?'},
   {'from': 'gpt', 'value': 'Yes'}],
  'bboxes': [[1, 52, 238, 159],
   [115, 78, 75, 67],
   [5, 59, 217, 134],
   [61, 156, 26, 31]]})